# Module `functools` en Python : `lru_cache`, `partial`, `cmp_to_key`

Dans cette section, nous allons explorer le module **`functools`**, une bibliothèque standard Python qui fournit des outils pour travailler avec des fonctions et des objets appelables. Nous nous concentrerons sur **`lru_cache`** pour la mise en cache, **`partial`** pour la spécialisation de fonctions, et **`cmp_to_key`** pour transformer des comparaisons.

## Qu’est-ce que `functools` ?
Le module `functools` offre des fonctions d’ordre supérieur qui améliorent la manipulation des fonctions, rendant le code plus efficace et flexible. Les trois outils que nous aborderons sont particulièrement utiles pour optimiser, personnaliser et adapter les comportements fonctionnels.

Commençons par `lru_cache` !

## Utilisation de `lru_cache`

La décorateur **`lru_cache`** (*Least Recently Used Cache*) met en cache les résultats d’une fonction pour éviter de recalculer les mêmes appels, améliorant les performances des fonctions récursives ou coûteuses.

### Importation
```python
from functools import lru_cache
```

### Fonctionnalités

- Mémorisation : Stocke les résultats en fonction des arguments.
- Paramètres :
    - maxsize : Taille maximale du cache (par défaut 128).
    - typed : Si True, distingue les types (ex. : 3 et 3.0).

### Exemple Simple

Optimisons une fonction récursive de Fibonacci.

In [19]:
from functools import lru_cache
import time

@lru_cache(maxsize=128)
def fibonacci(n: int) -> int:
    """Calcule le n-ième nombre de Fibonacci avec cache."""
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


In [20]:

# Test de performance
start = time.time()
print(fibonacci(35))  # Sortie : 9227465
print(f"Temps avec cache : {time.time() - start:.4f}s")  # Sortie : ~0.000Xs


9227465
Temps avec cache : 0.0001s


In [21]:

# Sans cache pour comparaison
def fibonacci_sans_cache(n: int) -> int:
    if n < 2:
        return n
    return fibonacci_sans_cache(n - 1) + fibonacci_sans_cache(n - 2)


In [22]:

start = time.time()
print(fibonacci_sans_cache(35))  # Sortie : 9227465
print(f"Temps sans cache : {time.time() - start:.4f}s")  # Sortie : ~3-5s (selon machine)

9227465
Temps sans cache : 1.1427s


## Analyse de `lru_cache`

- **Avec `lru_cache`** : Les résultats sont mis en cache, rendant les appels répétés instantanés après le premier calcul.
- **Sans cache** : Chaque appel recalcule tout, explosant en complexité (O(2^n) pour Fibonacci).
- **Cas d’usage** : Idéal pour les fonctions récursives coûteuses ou les appels fréquents avec les mêmes arguments (ex. : appels API, calculs mathématiques).

Passons à `partial` !

## Utilisation de `partial`

La fonction **`partial`** crée une nouvelle version d’une fonction avec certains arguments pré-remplis, simplifiant son appel ultérieur.

### Importation
```python
from functools import partial
```

### Fonctionnalités

- Fixe des arguments : Réduit le nombre de paramètres à fournir.
- Réutilisabilité : Crée des fonctions spécialisées à partir de génériques.

### Exemple Simple

Spécialisons une fonction de multiplication.

In [23]:
from functools import partial

def multiplier(x: int, y: int) -> int:
    """Multiplie deux nombres."""
    return x * y


In [24]:

# Création d’une version spécialisée
doubler = partial(multiplier, 2)


In [25]:
print(doubler(5))  


10


In [26]:
print(doubler(10))


20


In [27]:
# Avec plusieurs arguments
multiplier_par = partial(multiplier, y=3)
print(multiplier_par(4)) 

12


## Analyse de `partial`

- **`doubler`** : Une nouvelle fonction où `x` est fixé à 2, ne nécessitant qu’un seul argument.
- **Flexibilité** : Permet de personnaliser une fonction sans la redéfinir.
- **Cas d’usage** : Utile pour adapter des fonctions à des contextes spécifiques (ex. : callbacks avec arguments prédéfinis, configuration de paramètres par défaut).

Passons à `cmp_to_key` !

## Utilisation de `cmp_to_key`

La fonction **`cmp_to_key`** convertit une fonction de comparaison (style Python 2) en une clé de tri compatible avec `sorted()` ou `list.sort()`, qui utilisent des clés de tri depuis Python 3.

### Importation
```python
from functools import cmp_to_key
```

### Fonctionnalités

- Comparaison personnalisée : Transforme une fonction cmp(a, b) (retournant -1, 0, 1) en une clé de tri.
- Compatibilité : Remplace l’ancien paramètre cmp supprimé en Python 3.

### Exemple Simple

Trions une liste par longueur de chaînes.

In [28]:
from functools import cmp_to_key

def comparer_longueur(a: str, b: str) -> int:
    """Compare la longueur de deux chaînes : -1 si a < b, 0 si égal, 1 si a > b."""
    return -1 if len(a) < len(b) else (1 if len(a) > len(b) else 0)


In [29]:

# Liste à trier
chaines = ["chat", "chien", "éléphant", "rat"]


In [ ]:

# Tri avec cmp_to_key
trie = sorted(chaines, key=cmp_to_key(comparer_longueur))
print("Tri par longueur :", trie)


Tri par longueur : ['rat', 'chat', 'chien', 'éléphant']


In [ ]:
# Comparaison avec une clé directe
trie_key = sorted(chaines, key=len)
print("Tri avec key=len :", trie_key)

Tri avec key=len : ['rat', 'chat', 'chien', 'éléphant']


## Analyse de `cmp_to_key`

- **`comparer_longueur`** : Une fonction de comparaison explicite, utile pour des logiques complexes.
- **Vs `key`** : `key=len` est plus simple ici, mais `cmp_to_key` permet des comparaisons personnalisées impossibles avec une clé seule (ex. : ordre multi-critères).
- **Cas d’usage** : Idéal pour porter du code Python 2 ou pour des tris nécessitant une logique de comparaison explicite.

Voyons un exemple avancé combinant ces outils !

## Exemple Avancé : Traitement de Données

Combinons `lru_cache`, `partial`, et `cmp_to_key` pour traiter une liste de données.

### Exemple
Calculons des distances, spécialisons une fonction, et trions par ordre personnalisé.

In [32]:
from functools import lru_cache, partial, cmp_to_key
import math

# Fonction avec cache pour calculer une distance
@lru_cache(maxsize=32)
def distance(x1: int, y1: int, x2: int, y2: int) -> float:
    """Calcule la distance euclidienne entre deux points."""
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


In [33]:

# Spécialisation avec partial (origine fixée à (0,0))
distance_origine = partial(distance, 0, 0)

# Liste de points (x, y)
points = [(1, 1), (2, 2), (0, 1), (3, 0)]


In [34]:

# Calcul des distances
distances = [(x, y, distance_origine(x, y)) for x, y in points]
print("Distances :", distances)


Distances : [(1, 1, 1.4142135623730951), (2, 2, 2.8284271247461903), (0, 1, 1.0), (3, 0, 3.0)]


In [35]:

# Tri par distance décroissante avec cmp_to_key
def comparer_distance(a: tuple, b: tuple) -> int:
    """Compare les distances (index 2) dans l’ordre décroissant."""
    return 1 if a[2] < b[2] else (-1 if a[2] > b[2] else 0)


In [36]:

points_tries = sorted(distances, key=cmp_to_key(comparer_distance))
print("Points triés par distance décroissante :", points_tries)

Points triés par distance décroissante : [(3, 0, 3.0), (2, 2, 2.8284271247461903), (1, 1, 1.4142135623730951), (0, 1, 1.0)]


## Conclusion

Cette section vous a permis de maîtriser :
- **`lru_cache`** : Optimisation des performances par mise en cache, idéal pour les récursions coûteuses.
- **`partial`** : Spécialisation des fonctions pour simplifier leur réutilisation.
- **`cmp_to_key`** : Adaptation des comparaisons complexes pour le tri.

Le module `functools` enrichit vos outils pour manipuler les fonctions de manière efficace et élégante. Expérimentez avec ces outils pour optimiser et personnaliser vos programmes Python !